In [5]:
raw_documents = [
    "고양이가창밖을바라보고 있다.고양이는창밖을자주본다.",
    "강아지가 신나게 뛰어놀고 있다. 강아지는 활발하다.",
    "고양이와 강아지는 둘 다 귀엽다. 고양이와 강아지는 친구다.",
    "창밖에는 비가 내리고 있다. 비 오는 날은 분위기 있다.",
    "비 오는 날에는 따뜻한 차가 좋다. 따뜻한 차는 기분을 좋게 한다.",
    "강아지는 비 오는 날에도 밖에서 뛰어논다. 강아지는 비를 좋아한다.",
    "고양이는 주로 창가에서 낮잠을 잔다. 낮잠을 자는 고양이는 평화롭다."
]

In [3]:
import pandas as pd

#불용어 파일 가져오기
stopwords = pd.read_csv(r"D:\강의\텍스트마이닝(16시간)\텍스트마이닝코드\ko-stopwords.csv")
stopwords

,stopwords
0,가
1,가까스로
2,가령
3,각
4,각각
...,...
600,진짜
601,두
602,게
603,요


##### ♦️1. 불용어 사전을 사용한 def 함수를 만들어보자
  - 조건 : 키위로 스페이스 보정 / okt로 정규화 적용 및 어간추출 후 포스태깅 / 불용어사전적용 /'Noun' 'Verb' 'Adjective' 태그만 출력

In [9]:
from kiwipiepy import Kiwi #스페이스 보정을 위해 사용
kiwi = Kiwi()

from konlpy.tag import Okt #형태소 분리 할 때 사용
okt = Okt()

In [ ]:
#데이터 프레임인 불용어를 집합 형태로 변환해서 사용(리스트도 사용가능)
stopwords = set(stopwords['stopwords']) 

In [20]:
def pos_tagging(documents): #input data가 리스트
    result = []

    for doc in documents: 
        string = kiwi.space(doc) #띄어쓰기 보정
        pos_words = okt.pos(string, norm=True, stem=True) #포스태깅과 정규화, 어간추출 작업 진행 
        result_words = [word for word, tag in pos_words if tag in {'Noun', 'Verb', 'Adjective'} if word not in stopwords]
        result.append(result_words)
    return result

In [24]:
pos_document = pos_tagging(raw_documents)
pos_document

[['고양이', '창', '밖', '바라보다', '있다', '고양이', '창밖', '자주', '보다'],
 ['강아지', '신', '나다', '뛰어놀다', '있다', '강아지', '활발하다'],
 ['고양이', '강아지', '둘', '귀엽다', '고양이', '강아지', '친구'],
 ['창', '비', '내리다', '있다', '비', '오다', '날', '분위기', '있다'],
 ['비', '오다', '날', '따뜻하다', '차갑다', '좋다', '따뜻하다', '차다', '기분', '좋다', '하다'],
 ['강아지', '비', '오다', '날', '밖', '뛰다', '놓다', '강아지', '비', '좋아하다'],
 ['고양이', '주로', '창가', '낮잠', '자다', '낮잠', '자다', '고양이', '평화롭다']]

In [26]:
#CounterVectorizer의 데이터 형식을 맞춰주기 위해 join 사용
result_document = [' '.join(doc) for doc in pos_document] 
result_document

['고양이 창 밖 바라보다 있다 고양이 창밖 자주 보다',
 '강아지 신 나다 뛰어놀다 있다 강아지 활발하다',
 '고양이 강아지 둘 귀엽다 고양이 강아지 친구',
 '창 비 내리다 있다 비 오다 날 분위기 있다',
 '비 오다 날 따뜻하다 차갑다 좋다 따뜻하다 차다 기분 좋다 하다',
 '강아지 비 오다 날 밖 뛰다 놓다 강아지 비 좋아하다',
 '고양이 주로 창가 낮잠 자다 낮잠 자다 고양이 평화롭다']

## 1. BoW
[BOW의 한계점]
- 단어 순서, 즉 맥락을 고려하지 않음 → 의미가 달라질 수 있음
- 유사한 단어를 구별 하지못함 → "고양이"와 "고양이를"이 다른 단어로 인식됨

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

In [33]:
#CounterVectorizer 객체 생성
vectorizer = CountVectorizer()

In [35]:
#bow로 임베딩(=변환)할 데이터 넣기
bow = vectorizer.fit_transform(result_document)

In [39]:
#bow를 계산하는데 사용한 단어 리스트 출력하기
feature_name = vectorizer.get_feature_names_out()
feature_name

array(['강아지', '고양이', '귀엽다', '기분', '나다', '낮잠', '내리다', '놓다', '따뜻하다', '뛰다',
       '뛰어놀다', '바라보다', '보다', '분위기', '오다', '있다', '자다', '자주', '좋다', '좋아하다',
       '주로', '차갑다', '차다', '창가', '창밖', '친구', '평화롭다', '하다', '활발하다'],
      dtype=object)

In [41]:
#Bow 계산 값 출력하기
bow_array = bow.toarray()
bow_array

array([[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0],
       [2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1],
       [2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 1,
        1, 0, 0, 0, 0, 1, 0],
       [2, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0],
       [0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0,
        0, 1, 0, 0, 1, 0, 0]], dtype=int64)

In [43]:
bow_df = pd.DataFrame(bow_array, columns=feature_name)
bow_df

,강아지,고양이,귀엽다,기분,나다,낮잠,내리다,놓다,따뜻하다,뛰다,...,좋아하다,주로,차갑다,차다,창가,창밖,친구,평화롭다,하다,활발하다
0,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,2,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2,2,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,2,0,...,0,0,1,1,0,0,0,0,1,0
5,2,0,0,0,0,0,0,1,0,1,...,1,0,0,0,0,0,0,0,0,0
6,0,2,0,0,0,2,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0


## 2. 원-핫 인코딩
[원-핫 인코딩의 한계점]
- 차원의 저주
- 단어간의 관계 표현 및 의미 반영 불가능 (자동차↔차)
- 원핫 인코딩의 단점 : 희소 행렬(Sparse Matrix) 문제 → 데이터가 커지면 벡터가 너무 커짐

In [45]:
from sklearn.preprocessing import OneHotEncoder

In [47]:
feature_name

array(['강아지', '고양이', '귀엽다', '기분', '나다', '낮잠', '내리다', '놓다', '따뜻하다', '뛰다',
       '뛰어놀다', '바라보다', '보다', '분위기', '오다', '있다', '자다', '자주', '좋다', '좋아하다',
       '주로', '차갑다', '차다', '창가', '창밖', '친구', '평화롭다', '하다', '활발하다'],
      dtype=object)

In [49]:
feature_name.shape

(29,)

In [51]:
feature_name_onehot = feature_name.reshape(-1,1)
feature_name_onehot

array([['강아지'],
       ['고양이'],
       ['귀엽다'],
       ['기분'],
       ['나다'],
       ['낮잠'],
       ['내리다'],
       ['놓다'],
       ['따뜻하다'],
       ['뛰다'],
       ['뛰어놀다'],
       ['바라보다'],
       ['보다'],
       ['분위기'],
       ['오다'],
       ['있다'],
       ['자다'],
       ['자주'],
       ['좋다'],
       ['좋아하다'],
       ['주로'],
       ['차갑다'],
       ['차다'],
       ['창가'],
       ['창밖'],
       ['친구'],
       ['평화롭다'],
       ['하다'],
       ['활발하다']], dtype=object)

In [55]:
encoder = OneHotEncoder(sparse_output=False)
onehot_result = encoder.fit_transform(feature_name_onehot)

In [60]:
encoder.get_feature_names_out()

array(['x0_강아지', 'x0_고양이', 'x0_귀엽다', 'x0_기분', 'x0_나다', 'x0_낮잠', 'x0_내리다',
       'x0_놓다', 'x0_따뜻하다', 'x0_뛰다', 'x0_뛰어놀다', 'x0_바라보다', 'x0_보다',
       'x0_분위기', 'x0_오다', 'x0_있다', 'x0_자다', 'x0_자주', 'x0_좋다', 'x0_좋아하다',
       'x0_주로', 'x0_차갑다', 'x0_차다', 'x0_창가', 'x0_창밖', 'x0_친구', 'x0_평화롭다',
       'x0_하다', 'x0_활발하다'], dtype=object)

In [57]:
onehot_result

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0

In [100]:
#원핫인코딩된 값들을 보기좋게 key값과 value 값으로 매칭
#(기존 vectorizer를 사용했을때 사용자가 보기좋게 데이터 프레임으로 변환한 것과 같이 사전 형태로 변환)
one_hot_dict = dict(zip(encoder.get_feature_names_out(), onehot_result))
one_hot_dict

{'x0_강아지': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'x0_고양이': array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'x0_귀엽다': array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'x0_기분': array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'x0_나다': array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'x0_낮잠': array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'x0_내리다': array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

## 3. N-gram

### 3.1 bi-grams

In [62]:
from nltk import bigrams #두개씩 묶을때

In [64]:
pos_document

[['고양이', '창', '밖', '바라보다', '있다', '고양이', '창밖', '자주', '보다'],
 ['강아지', '신', '나다', '뛰어놀다', '있다', '강아지', '활발하다'],
 ['고양이', '강아지', '둘', '귀엽다', '고양이', '강아지', '친구'],
 ['창', '비', '내리다', '있다', '비', '오다', '날', '분위기', '있다'],
 ['비', '오다', '날', '따뜻하다', '차갑다', '좋다', '따뜻하다', '차다', '기분', '좋다', '하다'],
 ['강아지', '비', '오다', '날', '밖', '뛰다', '놓다', '강아지', '비', '좋아하다'],
 ['고양이', '주로', '창가', '낮잠', '자다', '낮잠', '자다', '고양이', '평화롭다']]

In [68]:
list(bigrams(pos_document[0]))

[('고양이', '창'),
 ('창', '밖'),
 ('밖', '바라보다'),
 ('바라보다', '있다'),
 ('있다', '고양이'),
 ('고양이', '창밖'),
 ('창밖', '자주'),
 ('자주', '보다')]

In [70]:
bigram_result = [list(bigrams(doc)) for doc in pos_document]
bigram_result

[[('고양이', '창'),
  ('창', '밖'),
  ('밖', '바라보다'),
  ('바라보다', '있다'),
  ('있다', '고양이'),
  ('고양이', '창밖'),
  ('창밖', '자주'),
  ('자주', '보다')],
 [('강아지', '신'),
  ('신', '나다'),
  ('나다', '뛰어놀다'),
  ('뛰어놀다', '있다'),
  ('있다', '강아지'),
  ('강아지', '활발하다')],
 [('고양이', '강아지'),
  ('강아지', '둘'),
  ('둘', '귀엽다'),
  ('귀엽다', '고양이'),
  ('고양이', '강아지'),
  ('강아지', '친구')],
 [('창', '비'),
  ('비', '내리다'),
  ('내리다', '있다'),
  ('있다', '비'),
  ('비', '오다'),
  ('오다', '날'),
  ('날', '분위기'),
  ('분위기', '있다')],
 [('비', '오다'),
  ('오다', '날'),
  ('날', '따뜻하다'),
  ('따뜻하다', '차갑다'),
  ('차갑다', '좋다'),
  ('좋다', '따뜻하다'),
  ('따뜻하다', '차다'),
  ('차다', '기분'),
  ('기분', '좋다'),
  ('좋다', '하다')],
 [('강아지', '비'),
  ('비', '오다'),
  ('오다', '날'),
  ('날', '밖'),
  ('밖', '뛰다'),
  ('뛰다', '놓다'),
  ('놓다', '강아지'),
  ('강아지', '비'),
  ('비', '좋아하다')],
 [('고양이', '주로'),
  ('주로', '창가'),
  ('창가', '낮잠'),
  ('낮잠', '자다'),
  ('자다', '낮잠'),
  ('낮잠', '자다'),
  ('자다', '고양이'),
  ('고양이', '평화롭다')]]

### 3.2 trigrams

In [75]:
from nltk import trigrams #세개씩 묶을때

In [80]:
list(trigrams(pos_document[0]))

[('고양이', '창', '밖'),
 ('창', '밖', '바라보다'),
 ('밖', '바라보다', '있다'),
 ('바라보다', '있다', '고양이'),
 ('있다', '고양이', '창밖'),
 ('고양이', '창밖', '자주'),
 ('창밖', '자주', '보다')]

In [82]:
trigram = [list(trigrams(doc)) for doc in pos_document]

In [84]:
trigram

[[('고양이', '창', '밖'),
  ('창', '밖', '바라보다'),
  ('밖', '바라보다', '있다'),
  ('바라보다', '있다', '고양이'),
  ('있다', '고양이', '창밖'),
  ('고양이', '창밖', '자주'),
  ('창밖', '자주', '보다')],
 [('강아지', '신', '나다'),
  ('신', '나다', '뛰어놀다'),
  ('나다', '뛰어놀다', '있다'),
  ('뛰어놀다', '있다', '강아지'),
  ('있다', '강아지', '활발하다')],
 [('고양이', '강아지', '둘'),
  ('강아지', '둘', '귀엽다'),
  ('둘', '귀엽다', '고양이'),
  ('귀엽다', '고양이', '강아지'),
  ('고양이', '강아지', '친구')],
 [('창', '비', '내리다'),
  ('비', '내리다', '있다'),
  ('내리다', '있다', '비'),
  ('있다', '비', '오다'),
  ('비', '오다', '날'),
  ('오다', '날', '분위기'),
  ('날', '분위기', '있다')],
 [('비', '오다', '날'),
  ('오다', '날', '따뜻하다'),
  ('날', '따뜻하다', '차갑다'),
  ('따뜻하다', '차갑다', '좋다'),
  ('차갑다', '좋다', '따뜻하다'),
  ('좋다', '따뜻하다', '차다'),
  ('따뜻하다', '차다', '기분'),
  ('차다', '기분', '좋다'),
  ('기분', '좋다', '하다')],
 [('강아지', '비', '오다'),
  ('비', '오다', '날'),
  ('오다', '날', '밖'),
  ('날', '밖', '뛰다'),
  ('밖', '뛰다', '놓다'),
  ('뛰다', '놓다', '강아지'),
  ('놓다', '강아지', '비'),
  ('강아지', '비', '좋아하다')],
 [('고양이', '주로', '창가'),
  ('주로', '창가', '낮잠'),
  ('창가', '낮잠', '자다'),


### 3.3 4-grams~

In [86]:
from nltk import ngrams #세개이상 묶을때

In [90]:
list(ngrams(pos_document[0], 4))

[('고양이', '창', '밖', '바라보다'),
 ('창', '밖', '바라보다', '있다'),
 ('밖', '바라보다', '있다', '고양이'),
 ('바라보다', '있다', '고양이', '창밖'),
 ('있다', '고양이', '창밖', '자주'),
 ('고양이', '창밖', '자주', '보다')]

In [98]:
ngrams_result = [list(ngrams(doc, 2)) for doc in pos_document]
ngrams_result

[[('고양이', '창'),
  ('창', '밖'),
  ('밖', '바라보다'),
  ('바라보다', '있다'),
  ('있다', '고양이'),
  ('고양이', '창밖'),
  ('창밖', '자주'),
  ('자주', '보다')],
 [('강아지', '신'),
  ('신', '나다'),
  ('나다', '뛰어놀다'),
  ('뛰어놀다', '있다'),
  ('있다', '강아지'),
  ('강아지', '활발하다')],
 [('고양이', '강아지'),
  ('강아지', '둘'),
  ('둘', '귀엽다'),
  ('귀엽다', '고양이'),
  ('고양이', '강아지'),
  ('강아지', '친구')],
 [('창', '비'),
  ('비', '내리다'),
  ('내리다', '있다'),
  ('있다', '비'),
  ('비', '오다'),
  ('오다', '날'),
  ('날', '분위기'),
  ('분위기', '있다')],
 [('비', '오다'),
  ('오다', '날'),
  ('날', '따뜻하다'),
  ('따뜻하다', '차갑다'),
  ('차갑다', '좋다'),
  ('좋다', '따뜻하다'),
  ('따뜻하다', '차다'),
  ('차다', '기분'),
  ('기분', '좋다'),
  ('좋다', '하다')],
 [('강아지', '비'),
  ('비', '오다'),
  ('오다', '날'),
  ('날', '밖'),
  ('밖', '뛰다'),
  ('뛰다', '놓다'),
  ('놓다', '강아지'),
  ('강아지', '비'),
  ('비', '좋아하다')],
 [('고양이', '주로'),
  ('주로', '창가'),
  ('창가', '낮잠'),
  ('낮잠', '자다'),
  ('자다', '낮잠'),
  ('낮잠', '자다'),
  ('자다', '고양이'),
  ('고양이', '평화롭다')]]